
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Create a ReAct Agent

In this lab, we will create a virtual stock advisor agent that uses the **ReAct** prompting framework. This agent will reason and act based on the prompts and tools provided.

Of course, the agent is built for demonstration purposes and the answers shouldn't be used as investment advice!


**Lab Outline:**

In this lab, you will need to complete the following tasks;

* **Task 1 :** Define the agent brain

* **Task 2 :** Define the agent tools

* **Task 3 :** Define an agent logic

* **Task 4 :** Create the agent 

* **Task 5 :** Run the agent

**📝 Your task:** Complete the **`<FILL_IN>`** sections in the code blocks and follow the other steps as instructed.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the lab, run the provided classroom setup scripts. These scripts will install the required libraries and configure variables which are needed for the lab.

In [0]:
%pip install -U -qq langchain-databricks langchain==0.3.7 langchain_community==0.3.5 wikipedia==1.4.0 youtube-search duckduckgo-search

dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-03LAB

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


2025-04-30 15:54:42,004 26627 ERROR _handle_rpc_error GRPC Error received
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/client/core.py", line 1721, in config
    resp = self._stub.Config(req, metadata=self.metadata())
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 277, in __call__
    response, ignored_call = self._with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 332, in _with_call
    return call.result(), call
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 439, in result
    raise self
  File "/databricks/python/lib/python3.10/site-packages/grpc/_interceptor.py", line 315, in continuation
    response, call = self._thunk(new_method).with_call(
  File "/databricks/python/lib/python3.10/site-packages/grpc/_channel.py", line 1193, in with_call
    return _end_unary_response_blocking(state, call, True, None)
 

**Enable MLflow Auto-Log**

In [0]:
import mlflow
mlflow.langchain.autolog()

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser10152510_1746020930@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser10152510_1746020930
Working Directory: /Volumes/dbacademy/ops/labuser10152510_1746020930@vocareum_com
Dataset Location:  NestedNamespace (dais='/Volumes/dbacademy_dais/v01')


## Task 1: Define the Brain of the Agent

Let's start by defining the agent brain using one of the available LLMs on Databricks.

* For this lab, use **databricks-meta-llama-3-3-70b-instruct** 

* Define the maximum number of tokens

In [0]:
from langchain.agents import AgentType, initialize_agent
from langchain_databricks import ChatDatabricks

# define the brain
llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)

/home/spark-6a681e8a-a99d-4bcc-9a97-64/.ipykernel/26627/command-3951983942472383-4136645084:5: LangChainDeprecationWarning: Use databricks_langchain.ChatDatabricks
  llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)


In [0]:
# let's test the brain
llm_llama.invoke("Hi! How are you?")

AIMessage(content="Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'prompt_tokens': 16, 'completion_tokens': 52, 'total_tokens': 68}, id='run-db7348f8-c795-4ba2-90e0-0660509fce5a-0')

Trace(request_id=tr-36e95bffe54d4d2e8e3387d263b38a37)

## Task 2 : Define Tools that the Agent Will Use

For an agent, the toolset is a crucial component to complete the defined task. The agent will reason about the next action to take and use an appropriate tool to complete that action.

For our stock advisor agent, we will use the following tools:

* **Wikipedia tool**: For searching company details. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/wikipedia/)]

* **DuckDuckGo Search tool**: This will be used for searching the latest news and updates about a stock.[[LangChain Doc](https://python.langchain.com/docs/integrations/tools/ddg/)]

* **YouTube search tool**: This will be used to search for review videos about the latest financial reports. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/youtube/)]

### Wikipedia Tool

In [0]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [0]:
# Test the Wikipedia tool
tool_wiki.run("Apple company history")

"Page: Apple Inc.\nSummary: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services. Founded in 1976 as Apple Computer Company by Steve Jobs, Steve Wozniak and Ronald Wayne, the company was incorporated by Jobs and Wozniak as Apple Computer, Inc. the following year. It was renamed Apple Inc. in 2007 as the company had expanded its focus from computers to consumer electronics. Apple is the largest technology company by revenue, with US$391.04 billion in the 2024 fiscal year.\nThe company was founded to produce and market Wozniak's Apple I personal computer. Its second computer, the Apple II, became a best seller as one of the first mass-produced microcomputers. Apple introduced the Lisa in 1983 and the Macintosh in 1984, as some of the first computers to use a graphical user interface and a mouse. By 1985, internal company problems led to Jo

Trace(request_id=tr-578f543a35924a1eb0d01e288039f115)

### DuckDuckGo Search Tool

In [0]:
# DuckDuckGo Search Tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

tool_news = DuckDuckGoSearchRun()

In [0]:
# Test DuckDuckGo tool
tool_news.run("Latest news about Apple stock")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-6a681e8a-a99d-4bcc-9a97-64714f559fb6/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


"Get the latest news on Apple stock with updates on market trends, financials, and analysis from Benzinga. Stay informed on AAPL performance Get a real-time Apple Inc. (AAPL) stock price with an overview of financials, statistics, forecasts, charts and more. ... The 12-month stock price forecast is $235.93, which is an increase of 12.27% from the latest price. Price Target. $235.93 (12.27% upside) Analyst Consensus: Buy. Stock Forecasts. News. All; ... Daily market news in bullet ... Prakash Investment Advisors LLC acquired a new stake in Apple Inc. (NASDAQ:AAPL - Free Report) in the 4th quarter, according to the company in its most recent filing with the SEC. The firm acquired 31,151 shares of the iPhone maker's stock, valued at approximately $7,801,000. Stay up-to-date on Apple Inc. Common Stock (AAPL) news with the latest updates, breaking headlines, news articles, and more from around the web at Nasdaq.com. Apple is scheduled to report fiscal second-quarter earnings after the closi

Trace(request_id=tr-d3c73647c95f4383b41d564e9e5342c9)

### YouTube Search Tool

In [0]:
from langchain_community.tools import YouTubeSearchTool

tool_youtube = YouTubeSearchTool()

In [0]:
# Test the YouTube search tool 
tool_youtube.run("Apple company stock")

"['https://www.youtube.com/watch?v=9fQwp4UHZHQ&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D', 'https://www.youtube.com/watch?v=-jICRX_YDUA&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D']"

Trace(request_id=tr-9f82eb16fa7b49ccbc0ad38fa63b91b4)

### Define Toolset

Next, add all tools together to be used in the next step.

In [0]:
tools = [tool_wiki, tool_youtube, tool_news]

## Task 3: Define Agent Logic

An agent requires a planning prompt that defines how it should **reason and act**. LangChain provides a few ready-to-use prompt templates through its **[LangChain Hub](https://smith.langchain.com/hub)**, including those that follow the **ReAct (Reason + Act)** style.

In this task, you'll pull a pre-defined ReAct-style prompt from LangChain Hub, which is designed to work with tool-using agents.

> 💡 Using the Hub simplifies your code and ensures you're working with prompt templates that are community-tested and regularly maintained.

📚 [Learn more: LangChain Agent Executor Documentation](https://python.langchain.com/docs/how_to/agent_executor/#create-the-agent)

In [0]:
# Import LangChain Hub
from langchain import hub

# Pull a ReAct-style prompt for function/tool-calling agents
prompt = hub.pull("hwchase17/openai-functions-agent")

# Display the prompt structure
prompt.messages

/local_disk0/.ephemeral_nfs/envs/pythonEnv-6a681e8a-a99d-4bcc-9a97-64714f559fb6/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Task 4: Create the Agent

The final step is to create the agent using the LLM, planning prompt and toolset.

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm_llama, tools, prompt)
agent_exec  = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Task 5: Run the Agent

Let's test the agent!

Note: YouTube search might not return the best results. For better results, you could use a paid API/Service that supports better search.

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

Format the final answer as HTML.
</blockquote>


In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

    Format the final answer as HTML.
    """})

displayHTML(response["output"])



> Entering new AgentExecutor chain...
<p>Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. As one of the world's largest and most successful companies, Apple has a strong track record of innovation and customer loyalty. However, it's essential to stay up-to-date with the latest news and trends affecting the company's stock performance.</p>

<p>To get an overview of the latest news about Apple, let's search for information on DuckDuckGo: <function=duckduckgo_search>{ "query": "Apple latest news" }</function>

> Finished chain.


Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. As one of the world's largest and most successful companies, Apple has a strong track record of innovation and customer loyalty. However, it's essential to stay up-to-date with the latest news and trends affecting the company's stock performance. 

 To get an overview of the latest news about Apple, let's search for information on DuckDuckGo: { "query": "Apple latest news" }

Trace(request_id=tr-e2532629fe97472281bb6d270ac09918)

It seems like the agent uses company name only when searching for videos. **Let's play with the prompt to get better results.**

In real-world, the agent would be used in a conversational way and we would just chat with the agent to tweak the search query or to adjust the response in a desired format. 

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

Write a nice summary of the results.

Format the final answer as HTML.
</blockquote>

In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.
    When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

    Write a nice summary of the results.

    Format the final answer as HTML.

    """})
displayHTML(response["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Apple latest news'}`




/local_disk0/.ephemeral_nfs/envs/pythonEnv-6a681e8a-a99d-4bcc-9a97-64714f559fb6/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


Craft is constantly adding new features to its notes and documents experience on the iPhone, iPad, Mac, Apple Vision Pro, and web. The latest addition is integration with Readwise, a service that ... CUPERTINO, CALIFORNIA Apple today announced its largest-ever spend commitment, with plans to spend and invest more than $500 billion in the U.S. over the next four years. This new pledge builds on Apple's long history of investing in American innovation and advanced high-skilled manufacturing, and will support a wide range of initiatives that focus on artificial intelligence, silicon ... Find news about Apple Company News in the Apple Newsroom archive. Access press releases, updates, photos, videos and more. Apple; ... Apple Services; Apple Stories; Search Newsroom Close. opens in new window. Latest News. Company News. PRESS RELEASE. Apple will spend more than $500 billion in the U.S. over the next four years. February 24, 2025 ... 7NEWS brings you the latest Apple news from Australia and 

Investing in Apple Stock: An Overview 
 Apple has recently announced its largest-ever spend commitment, with plans to spend and invest more than $500 billion in the U.S. over the next four years. This new pledge builds on Apple's long history of investing in American innovation and advanced high-skilled manufacturing, and will support a wide range of initiatives that focus on artificial intelligence, silicon, and more. 
 For those interested in investing in Apple stock, it's essential to stay up-to-date with the latest news and financials. A review of Apple's latest financials can be found on YouTube, with videos discussing the company's earnings, valuation, and investment opportunities. 
 Some recommended videos include:
 
 Apple Stock Finance Investment Earnings Valuation 
 Apple Stock Finance Investment Earnings Valuation 
 
 
 These videos provide an in-depth analysis of Apple's financial performance and offer insights into the company's future prospects. By staying informed and doing your own research, you can make a more informed decision about investing in Apple stock.

Trace(request_id=tr-313b095f6bf940a3b4aebb57e82b2913)


## Conclusion

In this lab, you successfully created a virtual stock advisor agent using the ReAct prompting framework. By defining the agent's brain, tools, logic, and execution flow, you demonstrated how the agent can interact with various data sources to provide investment insights.


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>